In [10]:
import pandas as pd
import numpy as np
import os
from functools import reduce

In [11]:
inputPath = "Subgingival plaque\JupyTool output collection/"
# "JupyTool output collection\JupyTool output collection/"
rankToMerge = 2     #1 = sk, 2 = k, 3 = p,...

outputName = "Subgingival plaque.tsv"
# "Supragingival plaque.tsv"
inputFileNames = os.listdir(inputPath)

In [12]:
tables = []
for file in inputFileNames:
    # Read out all Tables and remove unneccecary informations
    df = pd.read_csv(inputPath + file,sep="\t",header=1)
    df = df.iloc[: , 1:]
    if("taxid" in df.columns):
        df = df.drop(columns=["taxid"])
    df.columns = [file.replace(".tsv",""),"#KEY"]
    columns = df.columns.tolist()
    columns.reverse()
    df = df[columns]
    tables.append(df)
# Merge all tables into one
df = reduce(lambda df1,df2:pd.merge(df1,df2,on="#KEY",how="outer"),tables)
df = df.fillna(0)
df

,#KEY,MGYA00143469-SRS2301826,MGYA00143475-SRS2301830,MGYA00143477-SRS2301825,MGYA00143478-SRS2301829,MGYA00143482-SRS2301824,MGYA00143483-SRS2301827,MGYA00143486-SRS2301823,MGYA00143488-SRS2301828,MGYA00150693-SRS675025,...,MGYA00185713-ERS2517192,MGYA00185714-ERS2517179,MGYA00185715-ERS2517184,MGYA00185716-ERS2517225,MGYA00185717-ERS2517220,MGYA00185718-ERS2517218,MGYA00185719-ERS2517212,MGYA00185720-ERS2517200,MGYA00185721-ERS2517217,MGYA00185722-ERS2517221
0,sk__Bacteria,103433.0,108276.0,101341.0,158266.0,20534.0,22445.0,61336.0,237679.0,59170.0,...,4446.0,8147.0,5381.0,5666.0,9921.0,2380.0,2117.0,88515.0,5354.0,4390.0
1,sk__Bacteria;k__;p__;c__;o__;f__;g__;s__bacter...,7.0,0.0,2.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,sk__Bacteria;k__;p__;c__;o__;f__;g__;s__bacter...,1.0,1.0,1.0,0.0,0.0,3.0,4.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,sk__Bacteria;k__;p__;c__;o__;f__;g__;s__bacter...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,sk__Bacteria;k__;p__;c__;o__;f__;g__;s__bacter...,2.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2170,sk__Bacteria;k__;p__Firmicutes;c__Negativicute...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2171,sk__Bacteria;k__;p__Firmicutes;c__Clostridia;o...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2172,sk__Bacteria;k__;p__Bacteroidetes;c__Flavobact...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2173,sk__Bacteria;k__;p__Proteobacteria;c__Betaprot...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
# group rows by taxa level
df2 = df
df2 = df2.reset_index()
for index, row in df.iterrows():
    rowRaw = row['#KEY']
    rowSplit = rowRaw.split(";")
    if(len(rowSplit) >= rankToMerge):
        df2.loc[index,"#KEY"] = rowSplit[rankToMerge-1].replace("__","_taxa_")
    else:
        df2.loc[index,"#KEY"] = "noValue"

df2 = df2[df2["#KEY"] != "noValue"]
df2 = df2.groupby(["#KEY"]).sum()
df2 = df2.reset_index()
df2 = df2.drop(columns=["index"])
df2

,#KEY,MGYA00143469-SRS2301826,MGYA00143475-SRS2301830,MGYA00143477-SRS2301825,MGYA00143478-SRS2301829,MGYA00143482-SRS2301824,MGYA00143483-SRS2301827,MGYA00143486-SRS2301823,MGYA00143488-SRS2301828,MGYA00150693-SRS675025,...,MGYA00185713-ERS2517192,MGYA00185714-ERS2517179,MGYA00185715-ERS2517184,MGYA00185716-ERS2517225,MGYA00185717-ERS2517220,MGYA00185718-ERS2517218,MGYA00185719-ERS2517212,MGYA00185720-ERS2517200,MGYA00185721-ERS2517217,MGYA00185722-ERS2517221
0,k_taxa_,3417002.0,3654410.0,3553948.0,3657227.0,3791367.0,4067515.0,4295568.0,4021034.0,725559.0,...,526833.0,1138270.0,1214723.0,907018.0,1336439.0,535894.0,1039737.0,1121982.0,1402493.0,1281090.0
1,k_taxa_Fungi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,k_taxa_Viridiplantae,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0


In [14]:
# merge both Tables into one and save
result = pd.concat([df2,df])
result.to_csv(outputName,sep="\t",index=False)



# #KEY	anstelle von taxonomy?